In [8]:
from openai import OpenAI
import json, os, sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

### Load answers

In [9]:
source_filename = "../data/processed/UN_data.json"
des_filename = "../data/processed/UN_data_with_narrative_answers.json"

# Load articles
with open(source_filename, 'r', encoding='utf-8') as f:
    articles = json.load(f)

# Load questions
with open('../data/processed/narrative_questions.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

#print number of questions
print("Number of questions:", len(questions))

# Set API key
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

Number of questions: 6


In [3]:
def get_answers(article, questions, model):
    article_content = article.get('text')
    print(f"Proceeding article")
    pre = "You are an AI trained to understand speeches and generate concise answers to multiple-choice questions based on the content. \
        Please read the following speech carefully. After reading, answer ALL the questions listed below. \
            Your answers must be in capital letters and formatted as a single string, where each question number is followed by its corresponding answer letter. \
                Separate each question-answer pair with a semicolon. \
                    Example format: 1A;2B;3C;4D;... \n\n"

    prompt = pre + f"Speech Content:\n{article_content}\n\n Questions: {questions}\n"
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
    except Exception as e:
        print(f"Error: {e}")
    return response

In [ ]:
def convert_string_to_dict(answer_string):
    if answer_string == "":
        return {}
    if answer_string[-1] == ';':
        answer_string = answer_string[:-1]
    return {item[:-1]: item[-1] for item in answer_string.split(';')}

results = []
iteration = 0
max_iteration = len(articles)

for article in articles:

    answers = get_answers(article, questions, model="gpt-4o")

    pure_answers = answers.choices[0].message.content
    print(pure_answers)

    # convert string to dictionary 1A;2B;3C;44D to {'1': 'A', '2': 'B', '3': 'C', '44': 'D'}
    processed_answers = convert_string_to_dict(pure_answers)
    article['Narrative_answers']= processed_answers

    results.append(article)
    iteration += 1

# Save results to a JSON file
with open(des_filename, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

In [4]:
updated_json = "../data/processed/UN_data_with_narrative_answers.json"
with open(updated_json, 'r', encoding='utf-8') as f:
    articles = json.load(f)

len(articles)

1138

In [5]:
#give each article a unique number from 0 to 1137 and update the json
for i, article in enumerate(articles):
    article['id'] = i

with open(updated_json, 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

# Load articles
with open(updated_json, 'r', encoding='utf-8') as f:
    articles = json.load(f)

In [6]:
# Load articles
with open(updated_json, 'r', encoding='utf-8') as f:
    articles = json.load(f)

In [7]:
def check_narrative_format(narrative_answers):
    expected_keys = {"1", "2", "3", "4", "5", "6"}
    # Check if the keys match the expected format
    if isinstance(narrative_answers, dict):
        return set(narrative_answers.keys()) == expected_keys
    return False

# Iterate through articles and print session number and country if format doesn't match
for article in articles:
    narrative_answers = article.get("Narrative_answers", {})
    session_number = article.get("id", "N/A")
    country = article.get("country", "N/A")
    if not check_narrative_format(narrative_answers):
        print(f"Session Number: {session_number}, Country: {country}")